In [1]:
import random
import json_lines
from langdetect import detect_langs
import nltk
from nltk.stem import WordNetLemmatizer
import gensim
from sklearn.model_selection import train_test_split

LIMIT_INPUT_ROWS= 100
MIN_ENGLISH= 0.0
WNL = WordNetLemmatizer()
ALL_TAGS= ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
ACCEPTABLE_TAGS= ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
MIN_WORD_OCCURRENCES= 5

number_entries= 0
with open ('reviews.json', 'rb') as f:
    for item in json_lines.reader(f):
        number_entries+= 1

In [2]:
# READ IN INFO
reviews= []; polarity= []; are_early= []
included_indexs= []
j= 0
while j < LIMIT_INPUT_ROWS:
    randint= random.randint(0, number_entries-1)
    if not randint in included_indexs:
        included_indexs.append(randint)
        j+= 1
    
with open('reviews.json', 'rb') as f: # 5,000 entries in reviews.json
    for counter, item in enumerate(json_lines.reader(f)):
        if counter in included_indexs:
            reviews.append(item['text'])
            polarity.append(item['voted_up'])
            are_early.append(item['early_access'])

# END PRODUCTS: reviews, polarity, are_early

In [3]:
# FILTER TEXT BY LANGUAGE

filtered_reviews= []
filtered_polarity= []
filtered_are_early= []
for i, text in enumerate(reviews):
    try:
        langs = detect_langs(text)
    except:
        print("ERROR: langs failed to define itself. Happens occassionally - just rerun")
        pass
    for lang in langs:
        if str(lang)[0:2] == 'en':
            if float(str(lang)[3:]) > MIN_ENGLISH:
                filtered_reviews.append(reviews[i])
                filtered_polarity.append(polarity[i])
                filtered_are_early.append(are_early[i])
reviews= filtered_reviews
polarity= filtered_polarity
are_early= filtered_are_early

# END PRODUCTS: reviews, polarity, are_early

ERROR: langs failed to define itself. Happens occassionally - just rerun
ERROR: langs failed to define itself. Happens occassionally - just rerun
ERROR: langs failed to define itself. Happens occassionally - just rerun


In [4]:
# FILTER TEXT BY WORD TYPE

#print(reviews)
filtered_reviews= []
for i, text in enumerate(reviews):
    #print("-------------------------------------")
    new_text= []
    split_text= text.split()
    tags= nltk.pos_tag(split_text)
    #print(tags)
    #print("#############")
    lemd_split_text= []
    for word in split_text:
        lemd_split_text.append(WNL.lemmatize(word))
    tags= nltk.pos_tag(lemd_split_text)
    #print(tags)
    for j, word in enumerate(lemd_split_text):
        if tags[j][1] in ACCEPTABLE_TAGS:
            new_text.append(word)
    filtered_reviews.append(new_text)
reviews= [' '.join([str(elem) for elem in sublist]) for sublist in filtered_reviews]
print(reviews)

# END PRODUCTS: reviews, polarity, are_early

["The gun and crank in this area go through Trevor Phillips Enterprises... or they ain't going!", "One of the greatest GTA ever made. Great story and ton of stuff to do with many side quest and many storyline mission (~40h) to beat the main story and iconic character such a Big Smoke, CJ, Sweet, Ryder, the same looking cop and gangs, Grove Street, Ballas, Varrios Los Aztecas, Los Santos Vagos, San Fierro Rifa and many more. CJ return back to Los Santos to his mom's funeral, thing were a they use to be. CJ decides to stay in Los Santos, he re-builds his reputation and help Grove Street back to the top. Everything seems to go for them, then one day he find out Big Smoke and Ryder are with Ballas. Officer Tenpenny drive You out of the city into the countryside. There Cj meet new people and he travel to San Fierro he there meet Wu Zi Mu, Z and FBI Agent. He help them. The FBI agent kidnapped your brother Sweet and you are sent to the new land Las Venturas, (now all bridge have been unlocke

In [5]:
# GET REVIEWS AS LIST OF LISTS

no_sentences= 0
reviews_as_ll= [] # ll denoting the list of list data
for text in reviews:
    split_text= text.split()
    reviews_as_ll.append(split_text)
    no_sentences+= 1

# END PRODUCTS: ll_reviews, no_sentences

In [6]:
X_train, X_test, y_train, y_test= train_test_split(reviews_as_ll, polarity, test_size=0.2, random_state=42)

In [7]:
model= gensim.models.Word2Vec(min_count= MIN_WORD_OCCURRENCES)
model.build_vocab(reviews_as_ll)
model.train(reviews_as_ll, total_examples=no_sentences, epochs=model.epochs)  # can be a non-repeatable, 1-pass generator

(1692, 11795)

In [11]:
#model['gear']  # raw NumPy vector of a word
sims= model.wv.most_similar('the', topn=10)  # get other similar words
print(sims)

[('are', 0.4341857433319092), ('to', 0.41584882140159607), ('you', 0.41312918066978455), ('of', 0.3936181664466858), ('and', 0.3795650899410248), ('a', 0.3577914834022522), ('but', 0.3577427864074707), ('that', 0.3477071523666382), ('at', 0.34456679224967957), ('-There', 0.3431721031665802)]
